# Hyper-Parameter Tuning



In [5]:
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
iris = datasets.load_iris()
parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
svc = svm.SVC()
clf = GridSearchCV(svc, parameters)
clf.fit(iris.data, iris.target)


sorted(clf.cv_results_.keys())

#print(clf.cv_results_)

import pandas as pd

cv_results_df=pd.DataFrame(clf.cv_results_)
cv_results_df

mean_fit_time  std_fit_time  mean_score_time  std_score_time param_C  \
0       0.000637      0.000070         0.000377        0.000044       1   
1       0.000705      0.000040         0.000405        0.000051       1   
2       0.000609      0.000056         0.000402        0.000084      10   
3       0.000666      0.000104         0.000377        0.000020      10   

  param_kernel                         params  split0_test_score  \
0       linear   {'C': 1, 'kernel': 'linear'}           0.966667   
1          rbf      {'C': 1, 'kernel': 'rbf'}           0.966667   
2       linear  {'C': 10, 'kernel': 'linear'}           1.000000   
3          rbf     {'C': 10, 'kernel': 'rbf'}           0.966667   

   split1_test_score  split2_test_score  split3_test_score  split4_test_score  \
0           1.000000           0.966667           0.966667                1.0   
1           0.966667           0.966667           0.933333                1.0   
2           1.000000           0.900000           0.966667                1.0   
3           1.000000           0.966667           0.966667                1.0   

   mean_test_score  std_test_score  rank_test_score  
0         0.980000        0.016330                1  
1         0.966667        0.021082                4  
2         0.973333        0.038873                3  
3         0.980000        0.016330                1

In [ ]:
frwvunini

In [6]:
import numpy as np
import os
from keras.datasets import mnist
from keras.layers import *
from keras.models import *
from time import time

In [18]:
nb_classes = 10
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(X_train.shape[0], 784)
X_test = X_test.reshape(X_test.shape[0], 784)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

In [19]:
from keras.utils import np_utils
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

In [26]:
def create_model(optimizer='rmsprop'):
    model = Sequential()
    model.add(Dense(512, input_shape=(784,)))
    model.add(Activation('relu')) # An "activation" is just a non-linear function applied to the output
    model.add(Dropout(0.2))   # Dropout helps protect the model from memorizing or "overfitting" the training data
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(10))
    model.add(Activation('softmax')) # This special "softmax" a
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy']) 
    return model

In [29]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
start=time()
model = KerasClassifier(build_fn=create_model)
optimizers = ['rmsprop', 'adam']
init = ['glorot_uniform', 'normal', 'uniform']
epochs = [5, 10, 20]
batches = [5, 10, 20]
param_grid = dict(optimizer=optimizers, epochs=epochs, batch_size=batches)
grid = GridSearchCV(estimator=model, param_grid=param_grid)
grid_result = grid.fit(X_train, Y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
for params, mean_score, scores in grid_result.grid_scores_:
    print("%f (%f) with: %r" % (scores.mean(), scores.std(), params))
print("total time:",time()-start)

Epoch 1/5
6485/9600 [===================>..........] - ETA: 13s - loss: 0.0643 - accuracy: 0.9107

KeyboardInterrupt: 

In [ ]:
def create_model():
    model = Sequential()
    model.add(Dense(512, input_shape=(784,)))
    model.add(Activation('relu')) # An "activation" is just a non-linear function applied to the output
    model.add(Dropout(0.2))   # Dropout helps protect the model from memorizing or "overfitting" the training data
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(10))
    model.add(Activation('softmax')) # This special "softmax" a
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 
    return model

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.cross_validation import StratifiedKFold
from sklearn.cross_validation import cross_val_score
start=time()
np.random.seed(7)
model = KerasClassifier(build_fn=create_model, nb_epoch=20, batch_size=10,verbose=1)
kfold = StratifiedKFold(y=y_train,n_folds=10, shuffle=True, random_state=seed)
results = cross_val_score(model, X_train, Y_train, cv=kfold)
print("Smaller: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
print("total time:",time()-start)


In [ ]:
def create_model(optimizer='rmsprop', init='glorot_uniform'):
    model = Sequential()
    model.add(Dense(512, input_shape=(784,)))
    model.add(Activation('relu')) # An "activation" is just a non-linear function applied to the output
    model.add(Dropout(0.2))   # Dropout helps protect the model from memorizing or "overfitting" the training data
    model.add(Dense(512,init=init))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(10,init=init))
    model.add(Activation('softmax')) # This special "softmax" a
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy']) 
    return model
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.grid_search import GridSearchCV,RandomizedSearchCV
start= time()
model = KerasClassifier(build_fn=create_model,verbose=1)
n_iter_search = 16 # Number of parameter settings that are sampled.
optimizers = ['rmsprop', 'adam']
init = ['glorot_uniform', 'normal', 'uniform']
epochs = np.array([50, 100, 150])
batches = np.array([5, 10, 20])
param_grid = dict(optimizer=optimizers, nb_epoch=epochs, batch_size=batches, init=init)

random_search = RandomizedSearchCV(estimator=model, 
                                   param_distributions=param_grid,
                                   n_iter=n_iter_search)
random_search.fit(X_train, Y_train)
print("Best: %f using %s" % (random_search.best_score_, random_search.best_params_))
means = random_search.cv_results_['mean_test_score']
stds = random_search.cv_results_['std_test_score']
params = random_search.cv_results_['params']
print("total time:",time()-start)
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

## References

- [`GridSearchCV` documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV)
- [GridSearchCV with keras](https://www.kaggle.com/shujunge/gridsearchcv-with-keras)